## Generalizing PREPARE and SELECT

**Learning outcomes**

* Understand the action of SELECT and PREPARE oracles.

In [8]:
import numpy as np
import pennylane as qml

In the last node, we found a circuit for applying an arbitrary linear combination of unitaries. It pslit naturally into two subroutines, PREPARE, which prepared the state and coefficients, and SELECT which applied the unitaries. These were somewhat messily defined using nested circuit, but it turns there is a clean generalization of PREPARE and SELECT.

For a linear combination of unitaries:
$$\tilde{U} = \sum_{j} \alpha_j U_J $$
which $\alpha_j > 0$, we'll define PREPARE and SELECT as oracle (black boxes) with the following behaviour:
$$SELECT\Ket{j} \Ket{\psi} = \Ket{j}*U_J* \Ket{\psi} $$
$$PREPARE\Ket{0} = \frac{1}{\sqrt{\alpha}}\sum_{j} \sqrt{\alpha_j} \Ket{j}$$

For a linear combination of unitaries:
$$\tilde{U} = \sum_{j} \alpha_j U_J $$
which $\alpha_j > 0$, we'll define PREPARE and SELECT as oracle (black boxes) with the following behaviour:
$$SELECT\Ket{j} \Ket{\psi} = \Ket{j}*U_J* \Ket{\psi} $$
$$PREPARE\Ket{0} = \frac{1}{\sqrt{\alpha}}\sum_{j} \sqrt{\alpha_j} \Ket{j} = \Ket{\alpha}$$
,where $\alpha  = \sum_{j} \alpha_j$, and $\Ket{j}$ is an orthonormal basis for the auxiliary system with initial state $\Ket{0}$

Since we have only specified hot the PREPARE oracle acts on the state $\Ket{0}$, we have the freedom to chooose *any* unitary which perfoms this. It doesn't matter what it does to the stets orthogonal to $\Ket{0}$! A convenient  chhoice is the *Householder transformation*. For a state $\Ket{v}$, this is defined as
$$R_v = I - 2\Ket{v}\Braket{v}$$

**Codercise H.7.1.** Write a fuction for a creating the Householder transformation as a matrix, assuming the inout a normalized state. You will [np.outer](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) helpful.

In [7]:
def householder(state):
    """Create the matrix form of a Householder transformation.

    Args:
        state (array[complex]): A list of amplitudes representing a state.

    Returns:
        array[complex]: The matrix representation of the Householder transformation.
    """
    matrix = np.identity(2) - 2* np.outer(state,np.conjugate(state))
    return matrix
householder([0,1])

array([[ 1.,  0.],
       [ 0., -1.]])

We'll define the PREPARE oracke as the HOuseholder transformation of the state:
$$\Ket{0-\alpha} = \frac{\Ket{0} - \Ket{\alpha}}{\Ket{\Ket{0} - \Ket{\alpha}}}.$$
This requires a little explanation. First, $\Ket{\alpha}$ is a vector of coefficients which has been normalized and can therefore be interpreted as a state. Then $\Ket{0} - \Ket{\alpha}$ is a difference of state, and is normalized by the denominator.

**Codercise H.7.2** (a) Implement the PREPARE oracle using the Householder transformation; this is available from the previous exercise. For simplicity, we work with a sum of $2^k$ unitaries. This means our auxiliary register will have $k$ qubits, and the state $\Ket{0}$ is the all-zero state (usually denoted $\Ket{0}$).

In [11]:
k_bits = 3
zero_vec = np.array([1] + [0]*(2**k_bits - 1))
print(zero_vec)

[1 0 0 0 0 0 0 0]


In [15]:
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)

def PREPARE(alpha_list):
    """Create the PREPARE oracle as a matrix.

    Args:
        alpha_list (array[float]): A list of coefficients.

    Returns:
        array[complex]: The matrix representation of the PREPARE routine.
    """
    zero_vec = np.array([1] + [0]*(2**k_bits - 1))

    alpha_list = np.sqrt(alpha_list)/np.linalg.norm(alpha_list)
    numerator = zero_vec - alpha_list
    denominator = np.linalg.norm(numerator)
    state = numerator/denominator

    return householder(state)

(b) The circuit below shows a PREPARE oracle (red), followed by a SELECT oracle (blue), followed by PREPARE+ (red):
![circuit](./images/H.7.2.1.png)
if $\Ket{0}$ is measured on the auxuliary register, this circuit applies the linear combination $\tilde{U}$ to $\Ket(\psi)$. Implement the LCU circuit shown above. SELECT is provided for you.

In [ ]:
def SELECT(U_list):
    """Implement the SELECT oracle for 2^k unitaries."""
    for index in range(2**k_bits):
        ctrl_str = np.binary_repr(index, k_bits) # Create binary representation
        qml.ControlledQubitUnitary(U_list[index], control_wires=aux,
                                   wires=main, control_values=ctrl_str)

def LCU(alpha_list, U_list):
    """Implement LCU using PREPARE and SELECT oracles for 2^k unitaries.

    Args:
        alpha_list (list[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as
        complex arrays.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    pass
